## João Pedro Rodrigues Freitas - 11316552

In [272]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(0)

# TODO

- Fazer os mini batches (train, test)
- Fazer o one-hot
- Fazer normalização

# Data

In [273]:
class Data():
    def __init__(self, X: np.ndarray) -> None:
        self.data = X

    def getData(self) -> np.ndarray:
        return self.data

    def setTrainRatio(self, ratio: float) -> None:
        self.train_ratio = ratio

    def getTrainRatio(self) -> float:
        return self.train_ratio
    
    def getAttrSize(self) -> int:
        return self.data.shape[1]
    
    def shuffleData(self) -> None:
        self.data = np.random.permutation(self.data)

    def getTrainData(self) -> np.ndarray:
        return self.data[:int(self.data.shape[0] * self.train_ratio)]
    
    def getTestData(self) -> np.ndarray:
        return self.data[int(self.data.shape[0] * self.train_ratio):]

    def normalize(self) -> None:
        '''Normaliza cada atributo para o intervalo [0, 1]'''
        for i in range(self.getAttrSize()):
            self.data[:,i] = (self.data[:,i] - np.min(self.data[:,i])) / (np.max(self.data[:,i]) - np.min(self.data[:,i]))
    

## Autoencoder

In [274]:
class Autoencoder():
    
    def __init__(self, inputDim: int, hiddenLayers: int, neuronsPerLayer: list[int]) -> None:
        if len(neuronsPerLayer) != hiddenLayers:
            raise ValueError("The size of the list must be equal to the number of hidden layers")

        self.inputDim = inputDim
        self.hiddenLayers = hiddenLayers
        self.neuronsPerLayer = neuronsPerLayer
        self.W = []
        self.b = [] 
        self._initParams()

    def _initParams(self) -> None:
        # layersDim = [input, hidden1, hidden2 (latentSpace), hidden1, output]
        layersDim = [self.inputDim] + self.neuronsPerLayer + self.neuronsPerLayer[::-1][1:] + [self.inputDim]

        for i in range(1, len(layersDim)):
            # inicializa com distribuição uniforme entre -0.5 e 0.5
            self.W.append(np.random.rand(layersDim[i-1], layersDim[i]) - 0.5)
            self.b.append(np.random.rand(1, layersDim[i]) - 0.5)
    
    def _sigmoid(self, x: np.ndarray) -> np.ndarray:
        return 1 / (1 + np.exp(-x))
    
    def _der_sigmoid(self, x: np.ndarray) -> np.ndarray:
        return x * (1 - x)
    
    def forward(self):
        # Encoder
        for i in range(self.hiddenLayers):
            self.out.append(np.dot(self.ativ[i], self.W[i]) + self.b[i])
            self.ativ.append(self._sigmoid(self.out[i+1]))
             
        # Decoder
        for i in range(self.hiddenLayers, 2 * self.hiddenLayers):
            self.out.append(np.dot(self.ativ[i], self.W[i]) + self.b[i])
            self.ativ.append(self._sigmoid(self.out[i+1]))

        print(f'W_Shapes: {[w.shape for w in self.W]}')
        print(f'B_Shapes: {[b.shape for b in self.b]}')
        print(f'Out_Shapes: {[o.shape for o in self.out]}')
        print(f'Ativ_Shapes: {[a.shape for a in self.ativ]}')

    def backward(self, lr: float, n: int):
        err = 2 * (self.ativ[-1] - self.ativ[0]) # Erro da camada de saída
        # Backpropagation Decoder
        delta = err * self._der_sigmoid(self.out[-1]) # err * sigm'(Z)
        print("DELTA", delta.shape)

        for i in range(2 * self.hiddenLayers - 1, self.hiddenLayers - 1, -1):
            print(f'Ativ[{i}]: {self.ativ[i].shape}')
            dW = np.dot(delta.T, self.ativ[i]) / self.ativ[i].shape[0] # TODO: ou n?

            # TODO: axis 0 ou 1?
            db = np.sum(delta, axis=0, keepdims=True) / self.ativ[i].shape[0] # TODO: ou n?

            self.W[i] -= lr * dW.T
            self.b[i] -= lr * db.T

            delta = np.dot(self.W[i], delta) * self._der_sigmoid(self.out[i-1])

        # Backpropagation Encoder
        # delta = np.dot(self.W[self.hiddenLayers], delta) * self._der_sigmoid(self.out[self.hiddenLayers].T)

        for i in range(self.hiddenLayers - 1, -1, -1):
            dW = np.dot(delta, self.ativ[i]) / self.ativ[i].shape[0]
            db = np.sum(delta, axis=0, keepdims=True) / self.ativ[i].shape[0]

            self.W[i] -= lr * dW.T
            self.b[i] -= lr * db.T

            if i != 0:
                delta = np.dot(self.W[i], delta) * self._der_sigmoid(self.out[i-1].T)


    
    def fit(self, x, lr: float = 0.01, n_epochs: int = 100):
        for epoch in range(n_epochs):

            n = x.shape[0] # número de exemplos

            self.ativ = [x] # Ativacoes
            self.out = [x] # outputs, desconsiderar o primeiro

            ## Forward
            self.forward()
            self.backward(lr, n)

            if epoch % 10 == 0:
                print(f"Epoch: {epoch}")
                print(f"Loss: {np.mean(np.square(x - self.ativ[-1]))}")



In [275]:
def main() -> None:
    X = np.vstack([np.random.normal(1, 0.5, [100,10]),
                   np.random.normal(-2, 1, [100,10]),
                   np.random.normal(3, 0.75, [100,10])])

    data = Data(X)
    
    data.normalize()
    data.shuffleData()
    data.setTrainRatio(0.8)

    inputDim = data.getAttrSize()
    hiddenLayers = 2
    neuronsPerLayer = [5, 3] # Dimensão latente é a ultima camada oculta
    # [10, 5, 3, 5, 10]
    
    autoencoder = Autoencoder(inputDim, hiddenLayers, neuronsPerLayer)
    autoencoder.fit(data.getTrainData(), lr = 0.01, n_epochs = 1)

    # TODO: Testar o autoencoder

In [276]:
if __name__ == "__main__":
    main()

W_Shapes: [(10, 5), (5, 3), (3, 5), (5, 10)]
B_Shapes: [(1, 5), (1, 3), (1, 5), (1, 10)]
Out_Shapes: [(240, 10), (240, 5), (240, 3), (240, 5), (240, 10)]
Ativ_Shapes: [(240, 10), (240, 5), (240, 3), (240, 5), (240, 10)]
DELTA (240, 10)
Ativ[3]: (240, 5)


ValueError: non-broadcastable output operand with shape (1,10) doesn't match the broadcast shape (10,10)